### **Задание №1**. Предположим, что у вас в базе есть табличка с балансом по всем счетам клиента на конец дня. Из-за особенностей построения таблицы, часть значений незаполнена и физически это означает отсутствие информации об изменении баланса. Для простоты будем считать, пропущенные значения для одного пользователя не могут встретиться между двумя заполненными. Напишите SQL запрос для заполнения неизвестных значений по каждому клиенту последним известным.

client_id |date | value
--- | --- | ---
1 | 14.04.2020 | 10000
1 | 15.04.2020 | 5000
1 | 16.04.2020 | NaN
1 | 17.04.2020 | NaN
1 | 18.04.2020 | NaN
2 | 14.04.2020 | 250000
2 | 15.04.2020 | 250000
2 | 16.04.2020 | 230000
2 | 17.04.2020 | 230000
2 | 18.04.2020 | 225000
3 | 14.04.2020 | 50000
3 | 15.04.2020 | NaN
3 | 16.04.2020 | NaN
3 | 17.04.2020 | NaN
3 | 18.04.2020 | NaN


In [ ]:
-- создаем временную таблицу (из основной) в которой будут: дата 
-- последнего известного значения по каждому клиенту и значение
create temporary table tab1
select client_id, max(date), value
from sber 
where value is not null
group by client_id;
-- делаем update основной таблицы
update sber 
inner join tab1 
on sber.client_id=tab1.client_id and sber.value is null
set 
sber.value = tab1.value;
-- удаляем временную таблицу
drop table tab1;

### **Задание №1.1 (*)**. В прошлом задании мы считали, что пропущенные значения не могут находиться между двумя заполненными в рамках одного пользователя, что не всегда правда в реальных задачах. Напишите теперь SQL запрос без указанного допущения.

client_id |date | value
--- | --- | ---
1 | 14.04.2020 | 10000
1 | 15.04.2020 | 5000
1 | 16.04.2020 | NaN
1 | 17.04.2020 | 7000
1 | 18.04.2020 | NaN
2 | 14.04.2020 | 250000
2 | 15.04.2020 | 250000
2 | 16.04.2020 | 230000
2 | 17.04.2020 | 230000
2 | 18.04.2020 | 225000
3 | 14.04.2020 | 50000
3 | 15.04.2020 | NaN
3 | 16.04.2020 | NaN
3 | 17.04.2020 | 49000
3 | 18.04.2020 | NaN

In [ ]:
# <YOUR_CODE_HERE>

### **Задание №2**. Пусть клиенты обладают рядом статусов, закодированных цифрами: 0 - "улица", 1 - "зарплатник", 2 - "премиум", 3 - "пенсионер", 4 - "студент". У нас есть матрица с логами смен статусов за некоторый период, разверните ее в матрицу статус - статус с частотами перехода между ними.

Пример, как было:

client_id | prev_segment | new_segment|
--- | --- | ---
1 | 0 | 1
2 | 0 | 2
3 | 4 | 3

Пример, как нужно:

segment |0 | 1 | 2 | 3 | 4 
--- | --- | --- | --- | --- | ---
0 | 0 | 0.5 | 0.5 | 0 | 0 
1 | 0 |0 | 0 | 0 | 0
2 | 0 | 0 | 0 | 0 | 0
3 | 0 | 0 | 0 | 0 | 0
4 | 0 | 0 | 0 | 1 | 0

В процессе выполнения задания разрешается пользоваться библиотеками numpy и pandas.

In [2]:
import numpy as np
import pandas as pd

# Сгенерируем данные для задачи
data = pd.DataFrame(
    np.random.randint(low=0, high=5, size=(50000000, 2)),
    columns=['prev_segment', 'new_segment']
)

# <YOUR_CODE_HERE>
matrix = pd.DataFrame(np.zeros((5, 5)), columns=['to 0','to 1','to 2','to 3','to 4',])
shape=pd.Series(np.zeros((5)))
for i in range(5):
    for j in range(5):
# я бы все таки предположил что переход в тот же самый статус - какая-то нелепица, застрахуемся от таких случаев
        if i!=j:
            matrix.iloc[i,j]=data.query('prev_segment==@i and new_segment==@j').count()[0]
            shape[i]=shape[i]+data.query('prev_segment==@i and new_segment==@j').count()[0]
matrix=matrix.mul(1/shape, axis=0).round(2)
matrix.rename({0:"From 0", 1:"From 1",2:"From 2",3:"From 3",4:"From 4"}, inplace=True)
matrix


,to 0,to 1,to 2,to 3,to 4
From 0,0.00,0.25,0.25,0.25,0.25
From 1,0.25,0.00,0.25,0.25,0.25
From 2,0.25,0.25,0.00,0.25,0.25
From 3,0.25,0.25,0.25,0.00,0.25
From 4,0.25,0.25,0.25,0.25,0.00


### **Задание №2.1 (*)**. Если вы не пользовались библиотечными функциями и реализовывали алгоритм самостоятельно, напишите, пожалуйста, его асимптотическую сложность в терминах O-большое.

In [ ]:
# <YOUR_CODE_HERE>
# Предыдущий алгоритм реализован на 2 циклах (25  итераций)
# Этот же алгоритм реализован "честным" проходом по всему dataframe, время выполнения этого алгоритма
# в десятки раз (не стал дожидаться конца, проверил на меньших наборах данных) превышвет предыдущий.

matrix = pd.DataFrame(np.zeros((5, 5)), columns=['to 0','to 1','to 2','to 3','to 4',])
shape=pd.Series(np.zeros((5)))
for i in range(data.shape[0]):
# я бы все таки предположил что переход из статуса1 в, напрмер, статус1 какая-то нелепица, поэтому застрахуемся от таких случаев 
    if data.iloc[i,0]!=data.iloc[i,1]:
        matrix.iloc[data.iloc[i,0], data.iloc[i,1]]=matrix.iloc[data.iloc[i,0], data.iloc[i,1]]+1
        shape[data.iloc[i,0]]=shape[data.iloc[i,0]]+1
matrix=matrix.mul(1/shape, axis=0).round(2)
matrix.rename({0:"From 0", 1:"From 1",2:"From 2",3:"From 3",4:"From 4"}, inplace=True)
matrix

# Асимптотическая сложнось этого алгоритма - О(n)

### **Задание №3**. Постройте прогнозую модель для датасета с UCI Machine Learning: предобработайте категориальные фичи, отберите их, соберите схему валидации и отберите модели с помощью нее. Описание датасета по ссылке: https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29

Данная задача multilabel multiclass classification, поэтому предлагаю начать с одного из подходов:
- Свести задачу к multilabel binary classification, как это указано в описании к датасету;
- Свести задачку к binary classification тем или иным способом;
- Обучить множество multiclass/binary classification моделей;
- Что-то еще, что придет в голову.


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data

--2021-04-20 07:39:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346695 (339K) [application/x-httpd-php]
Saving to: ‘drug_consumption.data’

drug_consumption.da 100%[===================>] 338.57K  1.28MB/s    in 0.3s    

2021-04-20 07:39:48 (1.28 MB/s) - ‘drug_consumption.data’ saved [346695/346695]



In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('drug_consumption.data', header=None, index_col=0)
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [ ]:
# <YOUR_CODE_HERE>

### **Задание №4**. Предположим, что мы построили рекомендательную систему, и ожидаем, что средний чек клиента вследствие этого должен вырасти. Мы провели AB-тест и собрали данные по контрольной и тестовой группе. Предложите и проведите статистический тест, который можно применить к этим данным.

В процессе выполнения задания разрешено пользоваться библиотеками scipy и statsmodels.

In [5]:
import numpy as np

# Сгенерируем данные для задачи
control = np.random.lognormal(mean=5, sigma=2, size=(100000,))
test = np.random.lognormal(mean=5.04, sigma=2, size=(100000,))

# Выдвинем две гипотезы:
# HO - между группами наблюдений control и test нет статистически занчимых различий
# H1 - между группами наблюдений control и test есть статистически значимые различия
# Проведем оценку наших данных на предмет "нормальности", визуально видно - распределение данных категорически не похоже на нормальное,
# так же проведя тест Шапиро-Уилка видим, что нормальность к нам не имеет отношения (код не выкладываю, мне кажется в нем нет никакой хитрости).
# Распределение наших данных не является нормальным поэтому воспользуемся непараметрикой, тестом Манна-Уитни.

from scipy import stats
stats.mannwhitneyu(control, test)

# На уровне значимости 0,05 отвергаем Н0, т.к. p-value много меньше 0,05
# Утверждаем что между группами control и test есть статистически значимые различия.

MannwhitneyuResult(statistic=4938154307.0, pvalue=8.316869939010767e-07)

### **Задание №4 (*)**. В прошлом задании мы улавливали статистически значимое различие в средних в 0.04 единицы на выборке из 100000 клиентов. Насколько больше нам требуется собрать клиентов, чтобы уловить различие в 0.02?

In [11]:
# Расчитаем взвешенное стандартное отклонение для выборок (считаем что) одинакового размера
import math
s_pooled=math.sqrt((4+4)/2)
# Расчитаем размер эффекта
ES=round(0.02/s_pooled,4)
# Расчитаем азмер выборки необходимой для получения такого размера эффекта, с мощностью 0,8 и альфа=0,05
from statsmodels.stats.power import TTestIndPower
analysis = TTestIndPower()
size = analysis.solve_power(ES, power=0.8, alpha=0.05)
print('Для того что бы уловить различие в 0.02ед, с мощностью теста 0.8 и альфа=0.05 необходимо ', round(size), ' наблюдений.')

Для того что бы уловить различие в 0.02ед, с мощностью теста 0.8 и альфа=0.05 необходимо  156978  наблюдений.
